# HW2P2: Face Classification and Verification


Congrats on coming to the second homework in 11785: Introduction to Deep Learning. This homework significantly longer and tougher than the previous homework. You have 2 sub-parts as outlined below. Please start early! 


*   Face Recognition: You will be writing your own CNN model to tackle the problem of classification, consisting of 7000 identities
*   Face Verification: You use the model trained for classification to evaluate the quality of its feature embeddings, by comparing the similarity of known and unknown identities

For this HW, you only have to write code to implement your model architecture. Everything else has been provided for you, on the pretext that most of your time will be used up in developing the suitable model architecture for achieving satisfactory performance.

Common errors which you may face in this homeworks (because of the size of the model)


*   CUDA Out of Memory (OOM): You can tackle this problem by (1) Reducing the batch size (2) Calling `torch.cuda.empty_cache()` and `gc.collect()` (3) Finally restarting the runtime



# Preliminaries

In [ ]:
!nvidia-smi # to see what GPU you have

Thu Oct 27 22:04:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install wandb --quiet

     |████████████████████████████████| 1.9 MB 14.0 MB/s 
     |████████████████████████████████| 182 kB 72.0 MB/s 
     |████████████████████████████████| 166 kB 56.6 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 61.9 MB/s 
     |████████████████████████████████| 162 kB 56.8 MB/s 
     |████████████████████████████████| 162 kB 54.2 MB/s 
     |████████████████████████████████| 158 kB 49.8 MB/s 
     |████████████████████████████████| 157 kB 58.0 MB/s 
     |████████████████████████████████| 157 kB 55.2 MB/s 
     |████████████████████████████████| 157 kB 28.6 MB/s 
     |████████████████████████████████| 157 kB 52.1 MB/s 
     |████████████████████████████████| 157 kB 58.8 MB/s 
     |████████████████████████████████| 157 kB 62.7 MB/s 
     |████████████████████████████████| 157 kB 23.4 MB/s 
     |████████████████████████████████| 156 kB 55.6 MB/s 


In [ ]:
import torch
from torchsummary import summary
import torchvision #This library is used for image-based operations (Augmentations)
import torchvision.transforms as ttf
import os
import gc
from tqdm import tqdm
from PIL import Image
import multiprocessing
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import glob
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)
print(torch.cuda.get_device_name(0))
n_cpu = multiprocessing.cpu_count()
print(f"n_cpu = {n_cpu}")

Device:  cuda
Tesla T4
n_cpu = 2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import os.path as path 
# if not path.exists("/content/drive"):
#   !sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#   !sudo apt-get update -qq 2>&1 > /dev/null
#   !sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
#   !google-drive-ocamlfuse

#   !sudo apt-get install -qq w3m # to act as web browser 
#   !xdg-settings set default-web-browser w3m.desktop # to set default browser
#   %cd /content
#   !mkdir drive
#   %cd drive
#   !mkdir MyDrive
#   %cd ..
#   %cd ..
#   !google-drive-ocamlfuse /content/drive/MyDrive

In [ ]:
if not os.path.exists('/content/models'):
    os.mkdir(f"/content/models")
!cp /content/drive/MyDrive/CMU/TA/11785/hw2/hw2p2/models/* /content/models
if not os.path.exists('/content/tools'):
    os.mkdir(f"/content/tools")
!cp /content/drive/MyDrive/CMU/TA/11785/hw2/hw2p2/tools/* /content/tools
if not os.path.exists('/content/loss'):
    os.mkdir(f"/content/loss")
!cp /content/drive/MyDrive/CMU/TA/11785/hw2/hw2p2/loss/* /content/loss
if not os.path.exists('/content/trainer'):
    os.mkdir(f"/content/trainer")
!cp /content/drive/MyDrive/CMU/TA/11785/hw2/hw2p2/trainer/* /content/trainer

cp: -r not specified; omitting directory '/content/drive/MyDrive/CMU/TA/11785/hw2/hw2p2/models/__pycache__'
cp: -r not specified; omitting directory '/content/drive/MyDrive/CMU/TA/11785/hw2/hw2p2/tools/__pycache__'
cp: -r not specified; omitting directory '/content/drive/MyDrive/CMU/TA/11785/hw2/hw2p2/trainer/__pycache__'


In [ ]:
from models import ConvNeXt
from loss import CenterLoss
from trainer import HW2P2TrainerWithFineTuneLoss
from tools.load_model import load_model
from tools.kaggle import setup_kaggle

# TODOs
As you go, please read the code and keep an eye out for TODOs!

# Download Data from Kaggle

In [ ]:
setup_kaggle()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 5.4 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=e24bc5ca2059467361af8b941475398bf18b5cc3fc0a9fbb460f013d2473aee9
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!mkdir '/content/data'

!kaggle competitions download -c 11-785-f22-hw2p2-classification
!unzip -qo '11-785-f22-hw2p2-classification.zip' -d '/content/data'

!kaggle competitions download -c 11-785-f22-hw2p2-verification
!unzip -qo '11-785-f22-hw2p2-verification.zip' -d '/content/data'

 99% 2.34G/2.37G [00:25<00:00, 109MB/s]
100% 2.37G/2.37G [00:25<00:00, 100MB/s]
 54% 9.00M/16.8M [00:00<00:00, 29.8MB/s]
100% 16.8M/16.8M [00:00<00:00, 47.8MB/s]


# Configs

In [ ]:
import datetime

model_name = "convnext-T-CenterLoss"
checkpoints = '/content/model_checkpoints'
if not os.path.exists(checkpoints):
    os.mkdir(checkpoints)
drive_checkpoint = "/content/drive/MyDrive/CMU/TA/11785/hw2/hw2p2/model_checkpoints"
if not os.path.exists(drive_checkpoint):
    os.mkdir(drive_checkpoint)

date = datetime.datetime.now()
model_id = model_name + "_" + date.strftime("%m-%d_%H:%M:%S")
if not os.path.exists(f"{drive_checkpoint}/{model_id}"):
    os.mkdir(f"{drive_checkpoint}/{model_id}")

In [ ]:


config = {
    'batch_size': 128, # Increase this if your GPU can handle it
    'lr': 0.1,
    'epochs': 100,
    'weight_decay': 1e-4,
    'momentum': 0.9,
    'optimizer': 'SGD',
    'scheduler': 'CosineAnnealingLR',
    'loss_lr': 0.1,
    'loss_weight': 0.001, 
    'label_smoothing': 0.1, 
    'stochastic_depth_prob': [0, 0.1, 0.2, 0.2], 
    # 10 epochs is recommended ONLY for the early submission - you will have to train for much longer typically.
    # Include other parameters as needed.
}

# Classification Dataset

In [ ]:
DATA_DIR = '/content/data/11-785-f22-hw2p2-classification/'# TODO: Path where you have downloaded the data
TRAIN_DIR = os.path.join(DATA_DIR, "classification/train") 
VAL_DIR = os.path.join(DATA_DIR, "classification/dev")
TEST_DIR = os.path.join(DATA_DIR, "classification/test")

# Transforms using torchvision - Refer https://pytorch.org/vision/stable/transforms.html

train_transforms = ttf.Compose([ 
    # Implementing the right transforms/augmentation methods is key to improving performance.
                    ttf.RandomApply(transforms=[ttf.RandAugment()], p=0.1), 
                    ttf.RandomHorizontalFlip(p=0.5), 
                    ttf.RandomAdjustSharpness(sharpness_factor=2),
                    ttf.RandomApply(transforms=[ttf.ColorJitter(brightness=.3, hue=.1, saturation=0.2)], p=0.5), 
                    ttf.RandomResizedCrop((224, 224), scale=(0.25, 1.0)),
                    ttf.RandomApply(transforms=[ttf.RandomRotation(degrees=(-10, 10))], p=0.5), 
                    ttf.RandomPerspective(distortion_scale=0.4, p=0.75), 
                    ttf.ToTensor(), 
                    ])
# Most torchvision transforms are done on PIL images. So you convert it into a tensor at the end with ToTensor()
# But there are some transforms which are performed after ToTensor() : e.g - Normalization
# Normalization Tip - Do not blindly use normalization that is not suitable for this dataset

val_transforms = ttf.Compose([ttf.ToTensor()])


train_dataset = torchvision.datasets.ImageFolder(TRAIN_DIR, transform = train_transforms)
val_dataset = torchvision.datasets.ImageFolder(VAL_DIR, transform = val_transforms)
# You should NOT have data augmentation on the validation set. Why?


# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = config['batch_size'], 
                                           shuffle = True, drop_last=True, num_workers = n_cpu, pin_memory = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = config['batch_size'], 
                                         shuffle = False, num_workers = 2)

In [ ]:
# You can do this with ImageFolder as well, but it requires some tweaking
class ClassificationTestDataset(torch.utils.data.Dataset):

    def __init__(self, data_dir, transforms):
        self.data_dir   = data_dir
        self.transforms = transforms

        # This one-liner basically generates a sorted list of full paths to each image in the test directory
        self.img_paths  = list(map(lambda fname: os.path.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        return self.transforms(Image.open(self.img_paths[idx]))

In [ ]:
test_dataset = ClassificationTestDataset(TEST_DIR, transforms = val_transforms) #Why are we using val_transforms for Test Data?
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = config['batch_size'], shuffle = False,
                         drop_last = False, num_workers = 2)

In [ ]:
print("Number of classes: ", len(train_dataset.classes))
print("No. of train images: ", train_dataset.__len__())
print("Shape of image: ", train_dataset[0][0].shape)
print("Batch size: ", config['batch_size'])
print("Train batches: ", train_loader.__len__())
print("Val batches: ", val_loader.__len__())

Number of classes:  7000
No. of train images:  140000
Shape of image:  torch.Size([3, 224, 224])
Batch size:  128
Train batches:  1093
Val batches:  274


# Setup everything for training

In [ ]:
# model = FaceClassifier(num_classes=7000, normalize_embeddings=False).to(device)   
#model = ResNet50(num_classes=7000).to(device)
model = ConvNeXt(stochastic_depth_prob=config['stochastic_depth_prob']).to(device)

In [ ]:
num_trainable_parameters = 0
for p in model.parameters():
    num_trainable_parameters += p.numel()
print(num_trainable_parameters)

33189880


In [ ]:
criterion = torch.nn.CrossEntropyLoss(label_smoothing=config['label_smoothing'])# TODO: What loss do you need for a multi class classification problem?
fine_tuning_loss = CenterLoss(num_classes=7000, feat_dim=768, use_gpu=True)
optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9, weight_decay=config['weight_decay'])
optimizer_loss = torch.optim.SGD(fine_tuning_loss.parameters(), lr = config['loss_lr'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * config['epochs']))
# TODO: Implement a scheduler (Optional but Highly Recommended)
# You can try ReduceLRonPlateau, StepLR, MultistepLR, CosineAnnealing, etc.
scaler = torch.cuda.amp.GradScaler() # Good news. We have FP16 (Mixed precision training) implemented for you
# It is useful only in the case of compatible GPUs such as T4/V100

In [ ]:
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 56, 56]           4,704
       BatchNorm2d-2           [-1, 96, 56, 56]             192
            Conv2d-3           [-1, 96, 56, 56]           4,704
       BatchNorm2d-4           [-1, 96, 56, 56]             192
            Conv2d-5          [-1, 384, 56, 56]          37,248
              GELU-6          [-1, 384, 56, 56]               0
            Conv2d-7           [-1, 96, 56, 56]          36,960
   StochasticDepth-8           [-1, 96, 56, 56]               0
             Block-9           [-1, 96, 56, 56]               0
           Conv2d-10           [-1, 96, 56, 56]           4,704
      BatchNorm2d-11           [-1, 96, 56, 56]             192
           Conv2d-12          [-1, 384, 56, 56]          37,248
             GELU-13          [-1, 384, 56, 56]               0
           Conv2d-14           [-1, 96,

# Let's train!

In [ ]:
trainer = HW2P2TrainerWithFineTuneLoss(
    model_id, 
    config['batch_size'], 
    device, 
    checkpoints, 
    max_epochs = config['epochs'], 
    epoch_start = 0, 
    log_checkpoints = drive_checkpoint,
    model_saver_mode = 'max'
)
trainer.save_spec(str(model))

In [ ]:
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()

# Wandb

In [ ]:
wandb.login(key="<wandb-key>") #API Key is in your wandb account, under settings (wandb.ai/settings)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name = model_id, ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw2p2-ablations", ### Project should be created in your wandb account 
    config = config, ### Wandb Config for your run
    group = 'ConvNeXt'
)

wandb: Currently logged in as: ghensk. Use `wandb login --relogin` to force relogin


# Verification Task Setup

The verification task consists of the following generalized scenario:
- You are given X unknown identitites 
- You are given Y known identitites
- Your goal is to match X unknown identities to Y known identities.

We have given you a verification dataset, that consists of 1000 known identities, and 1000 unknown identities. The 1000 unknown identities are split into dev (200) and test (800). Your goal is to compare the unknown identities to the 1000 known identities and assign an identity to each image from the set of unknown identities. 

Your will use/finetune your model trained for classification to compare images between known and unknown identities using a similarity metric and assign labels to the unknown identities. 

This will judge your model's performance in terms of the quality of embeddings/features it generates on images/faces it has never seen during training for classification.

In [ ]:
known_regex = "/content/data/verification/known/*/*"
known_paths = [i.split('/')[-2] for i in sorted(glob.glob(known_regex))] 
# This obtains the list of known identities from the known folder

unknown_regex = "/content/data/verification/unknown_dev/*" #Change the directory accordingly for the test set

# We load the images from known and unknown folders
unknown_images = [Image.open(p) for p in tqdm(sorted(glob.glob(unknown_regex)))]
known_images = [Image.open(p) for p in tqdm(sorted(glob.glob(known_regex)))]

# Why do you need only ToTensor() here?
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()])

unknown_images = torch.stack([transforms(x) for x in unknown_images])
known_images  = torch.stack([transforms(y) for y in known_images ])
#Print your shapes here to understand what we have done

# You can use other similarity metrics like Euclidean Distance if you wish
similarity_metric = torch.nn.CosineSimilarity(dim= 1, eps= 1e-6) 

100%|██████████| 1000/1000 [00:00<00:00, 10822.19it/s]


# Experiments

In [ ]:
trainer.fit(model, train_loader, val_loader, optimizer, criterion, scheduler, scaler, fine_tuning_loss, optimizer_loss, config['loss_weight'], known_paths, unknown_images, known_images, similarity_metric, wandb, run)

Train Epoch 53/100::  95%|█████████▌| 1043/1093 [17:59<00:53,  1.06s/it, acc=0.0172%, fine_tune_loss=0.0290, lr=0.0454, num_correct=23]

In [ ]:
import shutil
for file in os.listdir(f"{checkpoints}/{model_id}"):
    src = f"{checkpoints}/{model_id}/" + file
    dest = f"{drive_checkpoint}/{model_id}/" + file
    shutil.copy(src, dest)

# Classification Task: Testing

In [ ]:
#Load your best model here if you need to
# checkpoint = torch.load('checkpoint.pth')
# model.load_state_dict(checkpoint['model_state_dict'])
# model = trainer.load_best_model()

In [ ]:
data = torch.load(f"{checkpoints}/{model_id}/epoch_98.model", map_location=device)
model = data.model

In [ ]:
# val_acc, val_loss = validate(model, val_loader, criterion)
# print("\nVal Acc {:.04f} Val Loss {:.04f}".format(val_acc, val_loss))

In [ ]:
def test(model,dataloader):

  model.eval()
  batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Test')
  test_results = []
  
  for i, (images) in enumerate(dataloader):
      images = images.to(device)

      with torch.inference_mode():
        outputs = model(images)

      outputs = torch.argmax(outputs, axis=1).detach().cpu().numpy().tolist()
      test_results.extend(outputs)
      
      batch_bar.update()
      
  batch_bar.close()
  return test_results

In [ ]:
test_results = test(model, test_loader)

## Generate csv to submit to Kaggle

In [ ]:
with open("classification_early_submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(test_dataset)):
        f.write("{},{}\n".format(str(i).zfill(6) + ".jpg", test_results[i]))

In [ ]:
src = "classification_early_submission.csv"
dest = f"{drive_checkpoint}/{model_id}/" + "classification_early_submission.csv"
shutil.copy(src, dest)

# Verification: Testing

In [ ]:
unknown_regex = "/content/data/verification/unknown_test/*" #Change the directory accordingly for the test set

# We load the images from known and unknown folders
unknown_images = [Image.open(p) for p in tqdm(sorted(glob.glob(unknown_regex)))]
unknown_images = torch.stack([transforms(x) for x in unknown_images])

In [ ]:
pred_id_strings = trainer.eval_verification(model, known_paths, unknown_images, known_images, similarity_metric, config['batch_size'], mode='test')

with open("verification_early_submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(pred_id_strings)):
        f.write("{},{}\n".format(i, pred_id_strings[i]))

In [ ]:
src = "verification_early_submission.csv"
dest = f"{drive_checkpoint}/{model_id}/" + "verification_early_submission.csv"
shutil.copy(src, dest)